# Chapter 28: Advanced Caching Strategies

Building upon real-time features, modern Next.js applications require sophisticated caching strategies to balance performance with data freshness. Multi-layer caching—from the edge to the client—can dramatically reduce latency and server load while maintaining consistency across global deployments.

By the end of this chapter, you'll master implementing multi-layer caching architectures, CDN integration strategies, edge caching patterns, cache invalidation techniques, stale-while-revalidate implementations, distributed caching with Redis, and cache performance monitoring.

## 28.1 Multi-Layer Caching Architecture

Implement a hierarchical caching strategy that leverages different storage layers based on data volatility and access patterns.

### Cache Layers Implementation

```typescript
// lib/cache/cache-layers.ts
import { LRUCache } from 'lru-cache';

// Layer 1: In-Memory (Fastest, process-specific)
const memoryCache = new LRUCache<string, any>({
  max: 500,
  ttl: 1000 * 60 * 5, // 5 minutes
  updateAgeOnGet: true,
  allowStale: false,
});

// Layer 2: Edge Cache (Vercel Edge Config or similar)
class EdgeCacheLayer {
  async get<T>(key: string): Promise<T | null> {
    try {
      const response = await fetch(`${process.env.EDGE_CONFIG_URL}/item/${key}`, {
        headers: { Authorization: `Bearer ${process.env.EDGE_CONFIG_TOKEN}` },
      });
      if (!response.ok) return null;
      return response.json();
    } catch {
      return null;
    }
  }

  async set(key: string, value: any, ttl?: number): Promise<void> {
    await fetch(process.env.EDGE_CONFIG_URL!, {
      method: 'PATCH',
      headers: {
        Authorization: `Bearer ${process.env.EDGE_CONFIG_TOKEN}`,
        'Content-Type': 'application/json',
      },
      body: JSON.stringify({
        items: [{ operation: 'upsert', key, value }],
      }),
    });
  }
}

// Layer 3: Redis/Distributed Cache
import { Redis } from '@upstash/redis';

const redis = new Redis({
  url: process.env.UPSTASH_REDIS_REST_URL!,
  token: process.env.UPSTASH_REDIS_REST_TOKEN!,
});

export class MultiLayerCache {
  private edge = new EdgeCacheLayer();

  async get<T>(key: string): Promise<T | null> {
    // Try memory first
    const memValue = memoryCache.get(key) as T;
    if (memValue) return memValue;

    // Try edge cache
    const edgeValue = await this.edge.get<T>(key);
    if (edgeValue) {
      memoryCache.set(key, edgeValue);
      return edgeValue;
    }

    // Try Redis
    const redisValue = await redis.get<T>(key);
    if (redisValue) {
      memoryCache.set(key, redisValue);
      await this.edge.set(key, redisValue);
      return redisValue;
    }

    return null;
  }

  async set<T>(key: string, value: T, options?: { 
    memoryTTL?: number; 
    edgeTTL?: number; 
    redisTTL?: number 
  }): Promise<void> {
    // Set in all layers
    memoryCache.set(key, value, { ttl: options?.memoryTTL });
    await Promise.all([
      this.edge.set(key, value, options?.edgeTTL),
      redis.set(key, value, options?.redisTTL ? { ex: options.redisTTL } : undefined),
    ]);
  }

  async invalidate(key: string): Promise<void> {
    memoryCache.delete(key);
    await Promise.all([
      this.edge.set(key, null),
      redis.del(key),
    ]);
  }

  async invalidatePattern(pattern: string): Promise<void> {
    // Clear memory cache matching pattern
    for (const key of memoryCache.keys()) {
      if (key.includes(pattern)) memoryCache.delete(key);
    }
    
    // Clear Redis keys matching pattern
    const keys = await redis.keys(`*${pattern}*`);
    if (keys.length > 0) {
      await redis.del(...keys);
    }
  }
}

export const cache = new MultiLayerCache();
```

### Cache-Aside Pattern with Next.js

Implement the cache-aside pattern for database queries:

```typescript
// lib/cache/cache-aside.ts
import { cache } from './cache-layers';

export async function getCachedData<T>(
  key: string,
  fetcher: () => Promise<T>,
  options?: {
    ttl?: number;
    staleWhileRevalidate?: number;
    tags?: string[];
  }
): Promise<T> {
  const cacheKey = `data:${key}`;
  const cached = await cache.get<T & { _timestamp: number }>(cacheKey);
  
  const now = Date.now();
  const ttl = options?.ttl || 3600;
  
  // Return cached data if fresh
  if (cached && (now - cached._timestamp) < ttl * 1000) {
    return cached as T;
  }
  
  // Return stale data while revalidating in background
  if (cached && options?.staleWhileRevalidate) {
    const staleThreshold = (ttl + options.staleWhileRevalidate) * 1000;
    if ((now - cached._timestamp) < staleThreshold) {
      // Trigger background revalidation
      revalidateInBackground(key, fetcher, options);
      return cached as T;
    }
  }
  
  // Fetch fresh data
  const data = await fetcher();
  await cache.set(cacheKey, { ...data, _timestamp: now }, {
    redisTTL: ttl,
  });
  
  return data;
}

async function revalidateInBackground<T>(
  key: string,
  fetcher: () => Promise<T>,
  options?: any
) {
  // Use waitUntil for edge runtime compatibility
  const { waitUntil } = require('@vercel/functions');
  
  waitUntil(
    (async () => {
      const data = await fetcher();
      await cache.set(`data:${key}`, { ...data, _timestamp: Date.now() }, {
        redisTTL: options?.ttl,
      });
    })()
  );
}
```

## 28.2 CDN Integration and Edge Caching

Configure Vercel Edge Network and other CDNs for optimal cache behavior.

### Edge Cache Configuration

```typescript
// app/api/content/[slug]/route.ts
import { NextRequest, NextResponse } from 'next/server';

export async function GET(
  request: NextRequest,
  { params }: { params: { slug: string } }
) {
  const { slug } = params;
  
  // Fetch content (this would hit your database/CMS)
  const content = await fetchContent(slug);
  
  if (!content) {
    return new NextResponse('Not Found', { status: 404 });
  }
  
  // Calculate cache headers based on content type
  const headers = new Headers();
  
  // Static content: Cache for 1 year, revalidate daily
  if (content.type === 'static') {
    headers.set('Cache-Control', 'public, max-age=31536000, s-maxage=86400, stale-while-revalidate=86400');
  }
  
  // Dynamic content: Cache for 5 minutes, serve stale for 1 hour
  else if (content.type === 'dynamic') {
    headers.set('Cache-Control', 'public, max-age=300, s-maxage=300, stale-while-revalidate=3600');
  }
  
  // Personalized content: Private, no CDN cache
  else if (content.type === 'personalized') {
    headers.set('Cache-Control', 'private, no-cache, no-store, max-age=0');
  }
  
  // Tag-based invalidation support
  headers.set('Cache-Tag', `content-${slug},content-type-${content.type}`);
  
  return NextResponse.json(content, { headers });
}

// lib/vercel/purge-cache.ts
export async function purgeCache(tags: string[]) {
  if (process.env.VERCEL_ENV !== 'production') return;
  
  await fetch('https://api.vercel.com/v1/edge-config/invalidations', {
    method: 'POST',
    headers: {
      Authorization: `Bearer ${process.env.VERCEL_TOKEN}`,
      'Content-Type': 'application/json',
    },
    body: JSON.stringify({
      tags,
      projectId: process.env.VERCEL_PROJECT_ID,
    }),
  });
}
```

### ISR with On-Demand Revalidation

Implement fine-grained revalidation strategies:

```typescript
// app/api/revalidate/route.ts
import { revalidateTag, revalidatePath } from 'next/cache';
import { NextRequest, NextResponse } from 'next/server';

export async function POST(request: NextRequest) {
  const { tag, path, secret } = await request.json();
  
  // Verify secret
  if (secret !== process.env.REVALIDATION_SECRET) {
    return NextResponse.json({ error: 'Invalid secret' }, { status: 401 });
  }
  
  try {
    if (tag) {
      revalidateTag(tag);
      console.log(`Revalidated tag: ${tag}`);
    }
    
    if (path) {
      revalidatePath(path);
      console.log(`Revalidated path: ${path}`);
    }
    
    return NextResponse.json({ revalidated: true, now: Date.now() });
  } catch (err) {
    return NextResponse.json(
      { error: 'Error revalidating' }, 
      { status: 500 }
    );
  }
}

// Usage in webhook handlers
// app/api/webhooks/cms/route.ts
export async function POST(req: Request) {
  const payload = await req.json();
  
  // Revalidate specific content
  await fetch(`${process.env.NEXT_PUBLIC_URL}/api/revalidate`, {
    method: 'POST',
    headers: { 'Content-Type': 'application/json' },
    body: JSON.stringify({
      tag: `content-${payload.documentId}`,
      secret: process.env.REVALIDATION_SECRET,
    }),
  });
  
  return Response.json({ success: true });
}
```

## 28.3 Redis for Distributed Caching

Implement Redis for cross-instance cache sharing and rate limiting.

### Redis Cache Implementation

```typescript
// lib/redis/cache.ts
import { Redis } from '@upstash/redis';
import { Ratelimit } from '@upstash/ratelimit';

const redis = new Redis({
  url: process.env.UPSTASH_REDIS_REST_URL!,
  token: process.env.UPSTASH_REDIS_REST_TOKEN!,
});

// Rate limiting with sliding window
export const ratelimit = new Ratelimit({
  redis,
  limiter: Ratelimit.slidingWindow(10, '10 s'),
  analytics: true,
});

export class RedisCache {
  async get<T>(key: string): Promise<T | null> {
    const data = await redis.get<T>(key);
    return data;
  }

  async set<T>(key: string, value: T, ttlSeconds?: number): Promise<void> {
    if (ttlSeconds) {
      await redis.setex(key, ttlSeconds, value);
    } else {
      await redis.set(key, value);
    }
  }

  async delete(key: string): Promise<void> {
    await redis.del(key);
  }

  async increment(key: string, amount: number = 1): Promise<number> {
    return await redis.incrby(key, amount);
  }

  // Bulk operations for efficiency
  async mget<T>(keys: string[]): Promise<(T | null)[]> {
    if (keys.length === 0) return [];
    return await redis.mget<T[]>(...keys);
  }

  async mset(entries: Record<string, any>): Promise<void> {
    const pipeline = redis.pipeline();
    Object.entries(entries).forEach(([key, value]) => {
      pipeline.set(key, value);
    });
    await pipeline.exec();
  }

  // Cache tagging for bulk invalidation
  async setWithTags(key: string, value: any, tags: string[], ttl?: number): Promise<void> {
    const multi = redis.multi();
    
    multi.set(key, JSON.stringify(value));
    if (ttl) multi.expire(key, ttl);
    
    // Add to tag sets
    tags.forEach(tag => {
      multi.sadd(`tag:${tag}`, key);
    });
    
    await multi.exec();
  }

  async invalidateTag(tag: string): Promise<void> {
    const keys = await redis.smembers(`tag:${tag}`);
    if (keys.length > 0) {
      const multi = redis.multi();
      keys.forEach(key => multi.del(key));
      multi.del(`tag:${tag}`);
      await multi.exec();
    }
  }
}

export const redisCache = new RedisCache();
```

## 28.4 Cache Invalidation Strategies

Implement predictable cache invalidation patterns.

### Tag-Based Invalidation

```typescript
// lib/cache/tagged-cache.ts
import { unstable_cache } from 'next/cache';

export const getProduct = unstable_cache(
  async (id: string) => {
    return await db.product.findUnique({ where: { id } });
  },
  ['product'],
  {
    tags: ['products'],
    revalidate: 3600,
  }
);

export const getProductList = unstable_cache(
  async (category: string) => {
    return await db.product.findMany({ where: { category } });
  },
  ['product-list'],
  {
    tags: ['products', `category-${category}`],
    revalidate: 1800,
  }
);

// Invalidation helpers
export async function invalidateProduct(productId: string) {
  'use server';
  
  revalidateTag('products');
  revalidateTag(`product-${productId}`);
}

export async function invalidateCategory(category: string) {
  'use server';
  
  revalidateTag(`category-${category}`);
  revalidateTag('products');
}
```

### Smart Cache Warming

Pre-populate cache after invalidation:

```typescript
// lib/cache/warmer.ts
import { redisCache } from './cache';

export class CacheWarmer {
  async warmProductCache(productId: string) {
    const product = await db.product.findUnique({
      where: { id: productId },
      include: { reviews: true, category: true },
    });
    
    if (!product) return;
    
    // Warm multiple cache layers
    await Promise.all([
      redisCache.set(`product:${productId}`, product, 3600),
      redisCache.set(`product:slug:${product.slug}`, product, 3600),
    ]);
    
    // Warm related caches
    if (product.category) {
      await this.warmCategoryCache(product.category.slug);
    }
  }

  private async warmCategoryCache(categorySlug: string) {
    const products = await db.product.findMany({
      where: { category: { slug: categorySlug } },
      take: 20,
    });
    
    await redisCache.set(`category:${categorySlug}`, products, 1800);
  }
}

export const cacheWarmer = new CacheWarmer();
```

## Key Takeaways from Chapter 28

1. **Multi-Layer Caching**: Implement a hierarchy from in-memory (L1) to edge cache (L2) to Redis (L3). Each layer serves different purposes: memory for hot data, edge for geographic distribution, Redis for shared state across instances.

2. **Cache-Control Headers**: Use granular cache headers—`s-maxage` for CDN, `max-age` for browser, and `stale-while-revalidate` for background updates. Tag responses with `Cache-Tag` for selective purging.

3. **ISR and On-Demand Revalidation**: Use `unstable_cache` for function-level caching with tags. Implement webhook-triggered revalidation for CMS-driven content, separating immediate stale-while-revalidate from manual purging.

4. **Redis Patterns**: Use Redis for distributed state, rate limiting (sliding window), and bulk operations. Implement tag-based invalidation using Redis Sets to track cache key relationships.

5. **Cache Warming**: After invalidation, asynchronously warm caches by fetching fresh data and populating all layers. This prevents cache stampedes when popular content expires.

## Coming Up Next

**Chapter 29: Progressive Web Applications**

Now that your application serves data efficiently through advanced caching, it's time to make it installable and offline-capable. In Chapter 29, we'll explore service workers, web app manifests, offline fallbacks, background sync, and push notifications to transform your Next.js app into a fully-featured PWA that works seamlessly even without network connectivity.